# Large Notebook 3: Reinfocement Learning
## Ex. 3: Control problem MountainCar-v0

In the exercise class we will cover the control problem of a car at the bottom of a valley which should pick-up enough momentum to get over the hill. We will use the environment from the OpenAI Gym, which allows you to play and visualize the 'game'. Use RL to train a policy that gets the car over the hill in the least amount of time. 

**Before you can start this exercise you have to install the package OpenAI Gym. Start your anaconda environment with python3 and install:**

* pip install gymnasium[classic-control]


In [ ]:
# UNCOMMENT IF GYMNASIUM FAILS TO IMPORT
#%pip install gymnasium[classic-control]

import gymnasium as gym
import numpy as np


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for gymnasium[classic-control] from https://files.pythonhosted.org/packages/c1/2a/f931b9b7515c16e5285ecb0e6c2d773266a4f781332f24456e7c0d516c8d/gymnasium-1.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpickle>=1.2.0 from https://files.pythonhosted.org/packages/7e/e8/64c37fadfc2816a7701fa8a6ed8d87327c7d54eacfbfb6edab14a2f2be75/cloudpickle-3.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for farama-notifications>=0.0.1 from https://files.pythonhosted.org/packages/05/2c/ffc08c54c05cdce6fbed2aeebc46348dbe180c6d2c541c7af7ba0aa5f5f8/Farama_Notifications-0.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for pygame>=2.1.3 from https://files.pythonhosted.org/packages/d2/55/ca3eb851aeef4f6f2e98a360c201f0d00bd1ba2eb98e2c7850d80aabc526/pygame-2.6.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.

If you have properly installed the openAI gym you should be able to import it. We will now run a DEMO to see if everything is working. The code already is able to simulate the MountainCar problem for the case that it actions are **random**. To be able to view the rendered video of the poor and helpless car, desperately trying to drive up the hill, you should run the code on your own computer.
For more info on this particular environment see e.g. the website: https://gymnasium.farama.org/environments/classic_control/mountain_car/



In [2]:
def demo():
    """run the MountainCar environment with random actions"""
    
    env = gym.make('MountainCar-v0', render_mode='human')  #  create an instance of the environment

    state = env.reset()  # reset the current game

    for _ in range(200):  # play 200 random actions
        env.render()  # render the current game state to screen
        a = env.action_space.sample()  # get a random action
        state, reward, terminated, truncated, info = env.step(a) # take the action and return the outcome
        
    env.close()
    
# run the demo
demo() 

## Building your RL player, ie. training your policy.
We have to start by creating the game environment and checking some properties of the state and action space:

In [ ]:
env = gym.make('MountainCar-v0')  # no rendering!

# get usefull information about the environment:
state = env.reset()
print('start state:', state)
print('Number of sctions in the action space:', env.action_space.n)
print('Lowest state in the state space:', env.observation_space.low)
print('Hightest state in the state space:', env.observation_space.high)

#perform one step of the game for action a=1
a=1
state, reward, terminated, truncated, info = env.step(a)
print('After the step with (a=1):',state, reward, terminated, truncated, info )


start state: (array([-0.5595893,  0.       ], dtype=float32), {})
Number of sctions in the action space: 3
Lowest state in the state space: [-1.2  -0.07]
Hightest state in the state space: [0.6  0.07]
After the step with (a=1): [-5.5931991e-01  2.6940505e-04] -1.0 False False {}


We see that the car starts out in state with two floats [-0.525, 0] (as it initializes random these numbers will differ each time you reset). You can perform any of 3 actions (a = 0 or 1 or 2). We don't know what the numbers in the state mean, they could be the $x$, $y$ coordinates of the car or the velocity and height, but **we also don't have to know!** We will let the RL algortithm learn how to drive the car regardless of the exact meaning of the state.

You should now code a function `s2q(s)` that links state `s` to a location in the Q-matrix. This can quickly be done by discretizing the state space into bins and determine the bin number corresponding to a certain value. The function should return a tuple (or list) `loc` that holds the two bin numbers.

In [37]:
def s2q(s):
    # convert continous state values to discrete location indices inside the Q matrix
    #----------ADD CODE HERE---------#
    print("state:",state)
    
    n_bins_pos = 10
    n_bins_vel = 10
    
    pos_min,pos_max = -1.2, 0.6
    vel_min,vel_max = -0.07,0.07
    
    # bin width for each position
    bin_width_pos = (pos_max-pos_min) /n_bins_pos
    bin_width_vel = (vel_max-vel_min) /n_bins_vel
    
    #pos, vel = s
    pos = s[0]
    vel = s[1]
    
    
    #bins indices
    print("pos",pos,"posmin",pos_min,"bin",bin_width_pos,"\n")
    pos_index = (pos-pos_min) / bin_width_pos
    print("posindex,",pos_index)
    pos_index = int(pos_index)
    vel_index = int((vel-vel_min) / bin_width_vel )
    
    #clipping
    position_index = min(max(pos_index, 0), n_bins_pos - 1)
    velocity_index = min(max(vel_index, 0), n_bins_vel - 1)

    loc = (position_index,velocity_index)
    
    
    return loc


The next function `qlearn()` should train your Q-matrix by playing `num_games` games according to an 'epsilon-greedy' strategy (Google it!) and update the Q-matrix accoding to the following Bellmann equation:

$$ \mathbf{Q}^{\rm new}[s_t,a_t]=(1-\alpha)\mathbf{Q}[s_t,a_t]+\alpha\left(R_t+\gamma\, \text{max}_a  \mathbf{Q}[s_{t+1},a]\right). $$

Here, $\alpha$ is the learning rate and $\gamma$ is the discount factor and are bounded by $\alpha,\gamma\in[0,1]$. These parameters have to be set with care, as they influence the speed of convergence of the Q-matrix. The discount factor lets you weigh the importance of future over immediate rewards. This is done by mixing-in the term $\text{max}_a  \mathbf{Q}[s_{t+1},a]$, which gives the maximum Q value in the future state.

In [ ]:
def qLearn(Q, α, γ, ϵ, ϵ_min, num_games):
    """ 
    learns the Q table by interacting with the environment and applying the Bellman eqation 
    Q: q-table (n-dimensional ndarray)
    α: learning rate
    γ: discount factor
    ϵ: probability of taking a random action in the epsilon-greedy policy
    ϵ_min: minimum value ϵ can take when applying a reduction algortihm to ϵ
    """

    state = env.reset()  # reset the current game
    wins = 0
    
    #----------ADD CODE HERE---------#
    for i in range(num_games):
        state = env.reset()
        done = False
        while not done:
            if np.random.uniform(0,1) < ϵ: #epsilon greedy algorithm
                action = env.action_space.sample() #take a random action
            # else choose the best know from the Q table
            else:
                s_index = s2q(state)
                action = np.argmax(Q[s_index])
            
            print(env.step(action))    
            next_state, reward, terminated, truncated, info  = env.step(action)
            
            next_s_index = s2q(next_state)
            s_index = s2q(state)
            
            #bellman equation
            best_next_action = np.max(Q[next_s_index])
            Q[s_index][action] = (1 - α) * Q[s_index][action] + α * (reward + γ * best_next_action)
            
            state=next_state
            
            if done and reward > 0:
                wins+=1
        
        if ϵ > ϵ_min:
            ϵ *= 0.99       
        
    print(f'Training ended. Number of wins: {wins}')

Finally you put everything together. It is almost completly finished for you. What values for the hyperparameters do you choose? 

In [39]:
# initialize the Q matrix as a numpy array with zeros
Qdim = (10,10,env.action_space.n)
Q = np.zeros(shape=Qdim)

# set the hyperparameters
α = 0.1  # learning rate [0,1]
γ = 0.90  # discount rate [0,1]
ϵ =  0.9 # epsilon greedy strategy [0,1]
ϵ_min = 0.1  # minimu value of epsilon [0,1]
num_games = 10 # number of training games

# train the agent and store results
qLearn(Q, α, γ, ϵ, ϵ_min, num_games)
np.save('qrun1.npy', Q)


state: [-5.5931991e-01  2.6940505e-04]
pos [-0.5228946  0.       ] posmin -1.2 bin 0.18 

posindex, [3.7616966 6.6666665]


TypeError: only length-1 arrays can be converted to Python scalars

Once a Q-matrix has been trained we can use it as a policy and play a game. Write code that performs actions according to the input Q-matrix to play a single episode.

In [ ]:
# replay the game using the trained Q matrix
Q = np.load('qrun1.npy')

# create and reset the environment with render mode on
env = gym.make('MountainCar-v0', render_mode='human')
state = env.reset()  
    
# play a single episode with max. 1000 actions
for _ in range(1000):           
    env.render()                
    loc = s2q(state)
    # a = np.argmax(Q[#----------ADD CODE HERE---------#]) 
    # state, reward, terminated, truncated, info = env.step(a) 
        
    if terminated: 
        print('Qplay Output:', reward, terminated, truncated, info)
        break

env.close()

# Experiments
Set up a couple fo experiments to figure oyt the following things:
* How do $\alpha$ and $\gamma$ effect your learning perfomance?
* Are both elements of the state vector equally important and can we reduce the dimensions of the Q-matrix of one (or both) of them?